In [1]:
# Import Python modules
import os, sys
import yaml
import numpy as np
import pandas as  pd
import xarray as xr
from datetime import timedelta, date
import textwrap

# matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.ticker as mticker

# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)

# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap
from ar_funcs import get_ar_days, duration_stats
from constants import ucsd_colors

ERROR 1: PROJ: proj_create_from_database: Open of /cw3e/mead/projects/cwp140/scratch/dnash/miniconda3/envs/SEAK-clim/share/proj failed


In [2]:
# Set up paths
server = "comet"
if server == "comet":
    path_to_data = '/cw3e/mead/projects/cwp140/scratch/dnash/data/'      # project data -- read only
elif server == "skyriver":
    path_to_data = '/work/dnash/data/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
# label strings
ivt_units = 'kg m$^{-1}$ s$^{-1}$'

# Select lat/lon grid
# bnds = [360-175., 360-120., 50., 60.] # extend of AK
bnds = [360-141., 360-130., 54., 61.] # extent of SEAK
ext1 = [-141.5, -130.0, 54., 61.5] # extent of SEAK
lonmin, lonmax, latmin, latmax = bnds

# set start and end dates to match WRF data
start_date = '2000-01-01 0:00'
end_date = '2019-12-31 23:00'

In [5]:
%%time
filename =  'downloads/AR_Catalog/globalARcatalog_ERA-Interim_1979-2019_v3.0.nc'
ar_filename = path_to_data + filename
ds = xr.open_dataset(ar_filename)
# ds = ds.assign_coords(lon=(((ds.lon + 180) % 360)-180))
# ds
da = ds.sel(time=slice(start_date, end_date), lat=slice(latmin, latmax), lon=slice(lonmin, lonmax))
da

CPU times: user 44.3 ms, sys: 10.3 ms, total: 54.7 ms
Wall time: 965 ms


<xarray.Dataset>
Dimensions:      (lon: 8, lat: 5, lev: 1, time: 28732, ens: 1)
Coordinates:
  * lon          (lon) float64 219.0 220.5 222.0 223.5 225.0 226.5 228.0 229.5
  * lat          (lat) float64 54.0 55.5 57.0 58.5 60.0
  * lev          (lev) float64 1e+03
  * time         (time) datetime64[ns] 2000-01-01 ... 2019-08-31T18:00:00
  * ens          (ens) float64 1.0
Data variables: (12/53)
    islnd        (lat, lon) float32 ...
    iscst        (lat, lon) float32 ...
    shapemap     (ens, time, lev, lat, lon) float32 ...
    axismap      (ens, time, lev, lat, lon) float64 ...
    tnsctmap     (ens, time, lev, lat, lon) float64 ...
    lflocmap     (ens, time, lev, lat, lon) float32 ...
    ...           ...
    axisidx      (ens, time, lev, lat, lon) float32 ...
    axislon      (ens, time, lev, lat, lon) float32 ...
    axislat      (ens, time, lev, lat, lon) float32 ...
    xtnsctidx    (ens, time, lev, lat, lon) float32 ...
    numobj       (ens, time, lev, lat, lon) float32 ...
    intblosto    (ens, lev, lat, lon) float64 ...
Attributes:
    title:                 Global Atmospheric River Database
    version:               2022.03.22
    creation_date:         22-Mar-2022 11:12:14
    tARget_version:        1.300
    dimension_convention:  5-d (nlon x nlat x nlev x ntime x nens) variables ...
    references:            (1) Guan, B., and D. E. Waliser (2015), Detection ...

In [6]:
## Option 1: AR days based on if they make "landfall" in SE AK
# option = 'landfall'

## Option 2: AR days based on if the AR is within SEAK bounding box
option = 'bounding-box'


In [7]:
lfloc = da.lflocmap.squeeze()
lfloc_ct = lfloc.count('time')
clim_freq = lfloc_ct.squeeze().values

index_clim = np.flatnonzero(clim_freq)
print(index_clim.shape)
landfall_clim = clim_freq.ravel()[index_clim]
print(landfall_clim.shape)

clim_lons, clim_lats = np.meshgrid(lfloc_ct.lon, lfloc_ct.lat)
clim_lat_list = clim_lats.ravel()[index_clim]
print(clim_lat_list.shape)
clim_lon_list = clim_lons.ravel()[index_clim]
print(clim_lon_list.shape)


(8,)
(8,)
(8,)
(8,)


In [8]:
## manually adding points to land mask
lats_add = [58.5, 58.5, 57., 57., 55.5, 55.5]
lons_add = [-139.5, -138.0, -136.5, -135., -135., -133.5]

for i, (xs1, ys1) in enumerate(zip(lons_add, lats_add)):
    da.islnd.loc[dict(lat=ys1, lon=xs1+360.)] = 1

In [9]:
lfdates = lfloc.where(lfloc > 0, drop=True).time.values # only pulls the dates where an AR made landfall in SE AK
print('No. of AR landfalls in SE AK:', len(lfdates))

kidmap = da.kidmap.squeeze()
kidmap = kidmap.where(da.islnd == 1) # mask kidmap where is ocean
ardates = kidmap.where(kidmap > 0, drop=True).time.values # pulls the dates when an AR object is within bbox 
print('No. of AR objects near SE AK:', len(ardates))

No. of AR landfalls in SE AK: 2546
No. of AR objects near SE AK: 5916


In [10]:
# create list of lists of trackIDs for each time step
if option == 'landfall':
    date_lst = lfdates
elif option == 'bounding-box':
    date_lst = ardates
final_lst = []
for i, ids in enumerate(date_lst):
#     print(ids)
    # pull track ID list for each time step
    x = kidmap.sel(time=ids).values.flatten()
    result = x[np.logical_not(np.isnan(x))]
    trackID = np.unique(result)
    # # get landfalling latitude and longitude
    # tmp = lfloc.sel(time=ids)
    # tmp = tmp.where(tmp > 0, drop=True)
    # lflat = tmp.lat.values
    # lflon = tmp.lon.values - 360.0 # to get degrees W value

    for j in range(len(trackID)):
        final_lst.append([ids, trackID[j]])
    
# put final_lst into df
track_df = pd.DataFrame(final_lst, columns=['date', 'trackID'])
track_df
track_ids = track_df.trackID.unique()
# create df with trackID, start date, end date, and duration of AR
data = []
for i, ids in enumerate(track_ids):
    idx = (track_df.trackID == ids)
    test = track_df.loc[idx]
    start = test.date.min()
    stop = test.date.max() + timedelta(hours=6)
    tmp = (stop-start)
    duration = tmp.total_seconds()/(3600) # convert to number of hours
    
    data.append([ids, start, stop, duration])
    
duration_df = pd.DataFrame(data, columns=['trackID', 'start_date', 'end_date', 'duration'])
duration_df

,trackID,start_date,end_date,duration
0,2.000011e+11,2000-01-06 00:00:00,2000-01-06 18:00:00,18.0
1,2.000011e+11,2000-01-08 00:00:00,2000-01-08 06:00:00,6.0
2,2.000012e+11,2000-01-19 06:00:00,2000-01-19 12:00:00,6.0
3,2.000012e+11,2000-01-27 06:00:00,2000-01-31 00:00:00,90.0
4,2.000013e+11,2000-01-28 18:00:00,2000-01-29 00:00:00,6.0
...,...,...,...,...
1495,2.019082e+11,2019-08-16 00:00:00,2019-08-21 12:00:00,132.0
1496,2.019082e+11,2019-08-21 12:00:00,2019-08-22 06:00:00,18.0
1497,2.019082e+11,2019-08-22 18:00:00,2019-08-23 00:00:00,6.0
1498,2.019082e+11,2019-08-23 12:00:00,2019-08-24 06:00:00,18.0


In [57]:
## save to csv
outfile = path_to_out + 'AR_track_duration_SEAK.csv'
duration_df.to_csv(outfile)

In [58]:
test = pd.read_csv(path_to_out + 'AR_track_duration_SEAK.csv')
test

,Unnamed: 0,trackID,start_date,end_date,duration
0,0,2.000011e+11,2000-01-06 00:00:00,2000-01-06 18:00:00,18.0
1,1,2.000011e+11,2000-01-08 00:00:00,2000-01-08 06:00:00,6.0
2,2,2.000012e+11,2000-01-19 06:00:00,2000-01-19 12:00:00,6.0
3,3,2.000012e+11,2000-01-27 06:00:00,2000-01-31 00:00:00,90.0
4,4,2.000013e+11,2000-01-28 18:00:00,2000-01-29 00:00:00,6.0
...,...,...,...,...,...
1495,1495,2.019082e+11,2019-08-16 00:00:00,2019-08-21 12:00:00,132.0
1496,1496,2.019082e+11,2019-08-21 12:00:00,2019-08-22 06:00:00,18.0
1497,1497,2.019082e+11,2019-08-22 18:00:00,2019-08-23 00:00:00,6.0
1498,1498,2.019082e+11,2019-08-23 12:00:00,2019-08-24 06:00:00,18.0


In [59]:
test.start_date

0       2000-01-06 00:00:00
1       2000-01-08 00:00:00
2       2000-01-19 06:00:00
3       2000-01-27 06:00:00
4       2000-01-28 18:00:00
               ...         
1495    2019-08-16 00:00:00
1496    2019-08-21 12:00:00
1497    2019-08-22 18:00:00
1498    2019-08-23 12:00:00
1499    2019-08-26 18:00:00
Name: start_date, Length: 1500, dtype: object

In [62]:
for index, row in test.iloc[1:3].iterrows():
    print(index, row)

1 Unnamed: 0                      1
trackID            200001050010.0
start_date    2000-01-08 00:00:00
end_date      2000-01-08 06:00:00
duration                      6.0
Name: 1, dtype: object
2 Unnamed: 0                      2
trackID            200001190611.0
start_date    2000-01-19 06:00:00
end_date      2000-01-19 12:00:00
duration                      6.0
Name: 2, dtype: object
